# Introduction:

This report is intended for analyze inflight catering invoices for a selected month compared to the previous month for domistic flights to answer the following questions:

1.	What is the Cost per Passenger(CPP)?
2.	What is the total pax?
3.	What is the number of pax for each class?
4.	What is the total expenses?
5.	What is the total expenses per class?
6.	What is the total load factor?
7.	What is the load factor per class? 
8. what is the total handling charges?
9. What are the monthly changes and their impact?

# INDEX:

## Table of Contents
<ul>
<li><a href="#numPAX">Number of Passengers:</a></li>
<li><a href="#seatCap">Seat Capacity:</a></li>
<li><a href="#lf">Load Factor:</a></li>
<li><a href="#yc_bev">YC beverage flight impact:</a></li>
<li><a href="#bc_bev">BC beverage flight impact:</a></li>
<li><a href="#yc_meal">YC meal flights impact:</a></li>
<li><a href="#bc_meal">BC meal flights impact:</a></li>
</ul>

## Data Cleaning:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from time import sleep
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Initiate Data Frame for the current month and previous month
df = pd.read_excel("AVIF Feb 2021.xlsb", engine='pyxlsb')
df_p = pd.read_excel("AVIF Jan 2021.xlsb", engine='pyxlsb')

In [4]:
del_col = ['MAIN_SBD_CODE', 'AIRCRAFT_TYPE', 'TOP5', 'ROUTE_TYPE', 'CUST_NO', 'ROUTE', 'SR_FLT_NO', 'SR_SUFFIX', 'SR_FROM_CITY', 'SR_TO_CITY', 'SERVICE_NO', 'FPM_CODE', 'MENU_CODE', 'MENU_DESC', 'QTY', 'VAT_PRICE', 'MENU_CYCLE']
df.drop(del_col,axis=1, inplace=True)
df_p.drop(del_col,axis=1, inplace=True)

In [5]:
# print first 5 rows of data
df.head()

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE
0,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,CMY2024,DISP. GOWNS (SV),PC,43016.0,100,0.00,0.00,C5
1,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,#HAGCA,#HAGCA-HANDLING CHARGE,NaN,36423.0,NaN,101.10,101.10,C5
2,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,BBSKTB1545,BBSKTB1545-BREAD ASSORTED HLD 3,MFOIL,39137.0,1-4/1,3.59,7.18,C5
3,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,DSYI1505,DSYI1505-CAKE BEE STING (80GR),PO,39182.0,100,2.88,17.28,C5
4,2110930001762,44228,1169,00:30,DMM,RUH,1,CA,6,6,ENYF1509,ENYF1509-HAMMOUR BAKED W/CARDINAL SAUCE,PO,39221.0,30,33.89,67.78,C5


In [6]:
df_p.head()

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE
0,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,CMY2024,DISP. GOWNS (SV),PC,43016,100,0.00,0.00,C5
1,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,#HAGCA,#HAGCA-HANDLING CHARGE,NaN,36423,NaN,101.10,101.10,C5
2,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,BBSKTB1544,BBSKTB1544-BREAD ASSORTED HLD 2,MFOIL,39143,1-4/1,5.21,10.42,C5
3,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,DSYI1504,DSYI1504-CAKE CHOCOLATE-BERRIES,PO,39186,100,4.78,28.68,C5
4,2010930015820,44197,1169,00:30,DMM,RUH,1,CA,6,6,ENYF1515,ENYF1515-HAMMOUR FILLET KAPSA,PO,39205,30,33.32,66.64,C5


In [7]:
# drop international flights colunn
df_p.drop(df_p[df_p['TYPE_CODE'] == 'C1'].index, inplace=True)
df.drop(df[df['TYPE_CODE'] == 'C1'].index, inplace=True)

In [8]:
# drop the not related class values to the passenger count
df.drop(df[df.CLASS_CODE.isin(['CA', 'CO', 'CP', 'PX', 'PXC','SM'])].index, inplace=True)
df_p.drop(df_p[df_p.CLASS_CODE.isin(['CA', 'CO', 'CP', 'PX', 'PXC','SM'])].index, inplace=True)

In [9]:
# drop rows with item descreption equals to LAVATORY DISINFECTANT 750ML (SV) 
df_p.drop(df_p[df_p['ITEM_DESCRIPTION'] == 'LAVATORY DISINFECTANT 750ML (SV)'].index, inplace=True)
df.drop(df[df['ITEM_DESCRIPTION'] == 'LAVATORY DISINFECTANT 750ML (SV)'].index, inplace=True)

In [10]:
# Convert the date to datetime64
df_p['DBL_DATE'] = pd.to_datetime(df_p['DBL_DATE'] - 25569, unit = 'D')

In [11]:
df['DBL_DATE']  = pd.to_datetime(df['DBL_DATE'] - 25569, unit = 'D')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166469 entries, 70 to 694544
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            166469 non-null  int64         
 1   DBL_DATE          166469 non-null  datetime64[ns]
 2   FLT_NO            166469 non-null  object        
 3   ETD               166469 non-null  object        
 4   FROM_CITY         166469 non-null  object        
 5   TO_CITY           166469 non-null  object        
 6   LEGNO             166469 non-null  int64         
 7   CLASS_CODE        166469 non-null  object        
 8   CAPACITY          166469 non-null  int64         
 9   NO_PAX            166469 non-null  int64         
 10  ITEM_CODE         166469 non-null  object        
 11  ITEM_DESCRIPTION  166469 non-null  object        
 12  UOM               161637 non-null  object        
 13  SPECS_NO          166468 non-null  float64       
 14  ALO

In [13]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164387 entries, 62 to 828410
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            164387 non-null  int64         
 1   DBL_DATE          164387 non-null  datetime64[ns]
 2   FLT_NO            164387 non-null  object        
 3   ETD               164387 non-null  object        
 4   FROM_CITY         164387 non-null  object        
 5   TO_CITY           164387 non-null  object        
 6   LEGNO             164387 non-null  int64         
 7   CLASS_CODE        164387 non-null  object        
 8   CAPACITY          164387 non-null  int64         
 9   NO_PAX            164387 non-null  int64         
 10  ITEM_CODE         164387 non-null  object        
 11  ITEM_DESCRIPTION  164387 non-null  object        
 12  UOM               158951 non-null  object        
 13  SPECS_NO          164387 non-null  int64         
 14  ALO

In [14]:
# Convert specs_no to string 
df_p['SPECS_NO'] = df_p['SPECS_NO'].astype(str)

In [15]:
df['SPECS_NO'].fillna(0, inplace=True)
df['SPECS_NO'] = df['SPECS_NO'].astype('int32')

In [16]:
df['SPECS_NO'] = df['SPECS_NO'].astype(str)

In [17]:
# Convert price to integer 
df_p['PRICE'] = df_p['PRICE'].astype(int)
df['PRICE'] = df['PRICE'].astype(int)

In [18]:
# change numerical columns to int32 and float32 to reduce memory usage 
df['NO_PAX'] = df['NO_PAX'].astype('int32')
df_p['NO_PAX'] = df_p['NO_PAX'].astype('int32')

df['CAPACITY'] = df['CAPACITY'].astype('int32')
df_p['CAPACITY'] = df_p['CAPACITY'].astype('int32')

df['UNIT_PRICE'] = df['UNIT_PRICE'].astype('float32')
df_p['UNIT_PRICE'] = df_p['UNIT_PRICE'].astype('float32')

In [19]:
# Change CDN_NO to String
df['CDN_NO'] = df['CDN_NO'].astype(str)
df_p['CDN_NO'] = df_p['CDN_NO'].astype(str)

In [20]:
# Change LEGNO to String
df['LEGNO'] = df['LEGNO'].astype(str)
df_p['LEGNO'] = df_p['LEGNO'].astype(str)

In [21]:
#Month variable
current_month = df.DBL_DATE.dt.month_name().iloc[0]
previous_month = df_p.DBL_DATE.dt.month_name().iloc[0]

In [22]:
#Check memory usage for df
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166469 entries, 70 to 694544
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            166469 non-null  object        
 1   DBL_DATE          166469 non-null  datetime64[ns]
 2   FLT_NO            166469 non-null  object        
 3   ETD               166469 non-null  object        
 4   FROM_CITY         166469 non-null  object        
 5   TO_CITY           166469 non-null  object        
 6   LEGNO             166469 non-null  object        
 7   CLASS_CODE        166469 non-null  object        
 8   CAPACITY          166469 non-null  int32         
 9   NO_PAX            166469 non-null  int32         
 10  ITEM_CODE         166469 non-null  object        
 11  ITEM_DESCRIPTION  166469 non-null  object        
 12  UOM               161637 non-null  object        
 13  SPECS_NO          166469 non-null  object        
 14  ALO

In [23]:
#Check memory usage for df_p
df_p.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164387 entries, 62 to 828410
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CDN_NO            164387 non-null  object        
 1   DBL_DATE          164387 non-null  datetime64[ns]
 2   FLT_NO            164387 non-null  object        
 3   ETD               164387 non-null  object        
 4   FROM_CITY         164387 non-null  object        
 5   TO_CITY           164387 non-null  object        
 6   LEGNO             164387 non-null  object        
 7   CLASS_CODE        164387 non-null  object        
 8   CAPACITY          164387 non-null  int32         
 9   NO_PAX            164387 non-null  int32         
 10  ITEM_CODE         164387 non-null  object        
 11  ITEM_DESCRIPTION  164387 non-null  object        
 12  UOM               158951 non-null  object        
 13  SPECS_NO          164387 non-null  object        
 14  ALO

### Import and Clean Summer Schedule: 

This step will help in categorizing flights as per meal policy

In [24]:
# import the summer schedule to get block time
summerSchedule = pd.read_excel('summer_schedule.xlsx')

In [25]:
# Print first five rows
summerSchedule.head()

,Flt Desg,Eff Date,Dis Date,Freq,Dept Arp,Dept Time,Arvl Arp,Arrv Time,Subfleet,Block Time
0,SV 0020,2021-06-03,2021-10-28,1..4...,JFK,18:00:00,JED,05:40:00,SV 77Z,11:40:00
1,SV 0021,2021-06-03,2021-10-28,1..4...,JED,02:55:00,JFK,15:50:00,SV 77Z,12:55:00
2,SV 0022,2021-06-01,2021-10-29,.2..5..,JFK,18:00:00,RUH,05:55:00,SV 77Z,11:55:00
3,SV 0023,2021-06-01,2021-06-29,.2..5..,RUH,02:25:00,JFK,15:50:00,SV 77Z,13:25:00
4,SV 0023,2021-07-02,2021-10-29,.2..5..,RUH,02:25:00,JFK,15:50:00,SV 77Z,13:25:00


In [26]:
# rename sector columns to be aligned with the other dataframe to be joined with 
summerSchedule.rename(columns={"Dept Arp": "FROM_CITY", "Arvl Arp": "TO_CITY", "Block Time": "BLOCK_TIME"}, inplace=True)

In [27]:
# drop unused columns
summerSchedule = summerSchedule.drop(columns=['Eff Date', 'Dis Date','Freq','Dept Time','Arrv Time','Subfleet', 'Flt Desg'])

In [28]:
# Print first five rows
summerSchedule.head()

,FROM_CITY,TO_CITY,BLOCK_TIME
0,JFK,JED,11:40:00
1,JED,JFK,12:55:00
2,JFK,RUH,11:55:00
3,RUH,JFK,13:25:00
4,RUH,JFK,13:25:00


In [29]:
# remove duplicates to ensure merge will become 1:m
summerSchedule.drop_duplicates(subset=['FROM_CITY', 'TO_CITY'], inplace=True)

In [30]:
# print new df informations
summerSchedule.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 3266
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FROM_CITY   318 non-null    object
 1   TO_CITY     318 non-null    object
 2   BLOCK_TIME  318 non-null    object
dtypes: object(3)
memory usage: 9.9+ KB


In [31]:
#perform the join of the dataframes
df = df.merge(summerSchedule, on=['FROM_CITY','TO_CITY'],how = 'inner')
df_p = df_p.merge(summerSchedule, on=['FROM_CITY','TO_CITY'],how = 'inner')

In [32]:
# Convert block time into minutes
df_p['BLOCK_TIME'] =  pd.to_datetime(df_p['BLOCK_TIME'].astype(str))
df['BLOCK_TIME'] =  pd.to_datetime(df['BLOCK_TIME'].astype(str))

In [33]:
df_p['BLOCK_TIME'] = df_p['BLOCK_TIME'].dt.hour*60+df_p['BLOCK_TIME'].dt.minute
df['BLOCK_TIME'] = df['BLOCK_TIME'].dt.hour*60+df['BLOCK_TIME'].dt.minute

## Create a paxFarme:

In [34]:
#Create a copy of the main data farme and store it in the variable df_copy
df_copy = df.copy()
df_p_copy = df_p.copy()

In [35]:
# make an instance of df_copy and call it paxFrame
paxFrame = df_copy[['CDN_NO', 'DBL_DATE', 'FLT_NO', 'TYPE_CODE', 'FROM_CITY', 'TO_CITY', 'CAPACITY', 'NO_PAX', 'CLASS_CODE', 'BLOCK_TIME']]
paxFrameP = df_p_copy[['CDN_NO', 'DBL_DATE', 'FLT_NO', 'TYPE_CODE', 'FROM_CITY', 'TO_CITY', 'CAPACITY', 'NO_PAX', 'CLASS_CODE', 'BLOCK_TIME']]

In [36]:
#check the information of paxFrame
paxFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166073 entries, 0 to 166072
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   CDN_NO      166073 non-null  object        
 1   DBL_DATE    166073 non-null  datetime64[ns]
 2   FLT_NO      166073 non-null  object        
 3   TYPE_CODE   166073 non-null  object        
 4   FROM_CITY   166073 non-null  object        
 5   TO_CITY     166073 non-null  object        
 6   CAPACITY    166073 non-null  int32         
 7   NO_PAX      166073 non-null  int32         
 8   CLASS_CODE  166073 non-null  object        
 9   BLOCK_TIME  166073 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 12.7+ MB


In [37]:
#check the information of paxFrameP
paxFrameP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163848 entries, 0 to 163847
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   CDN_NO      163848 non-null  object        
 1   DBL_DATE    163848 non-null  datetime64[ns]
 2   FLT_NO      163848 non-null  object        
 3   TYPE_CODE   163848 non-null  object        
 4   FROM_CITY   163848 non-null  object        
 5   TO_CITY     163848 non-null  object        
 6   CAPACITY    163848 non-null  int32         
 7   NO_PAX      163848 non-null  int32         
 8   CLASS_CODE  163848 non-null  object        
 9   BLOCK_TIME  163848 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 12.5+ MB


In [38]:
# Check the number of passengers
paxFrame['NO_PAX'].sum()

5278920

In [39]:
# Check the number of passengers
paxFrameP['NO_PAX'].sum()

5267624

In [40]:
# the number of passenger shown above is un realistic becuase of duplicated 'NO_PAX' which is related to menu items
# Therefore this code is used to remove duplicated rows
paxFrame = paxFrame.drop_duplicates(subset=['FROM_CITY', 'TO_CITY','DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'])

In [41]:
# the number of passenger shown above is un realistic becuase of duplicated 'NO_PAX' which is related to menu items
# Therefore this code is used to remove duplicated rows
paxFrameP = paxFrameP.drop_duplicates(subset=['FROM_CITY', 'TO_CITY', 'DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'])

In [42]:
# Check the paxFrame after removing duplicates
paxFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14105 entries, 0 to 166072
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CDN_NO      14105 non-null  object        
 1   DBL_DATE    14105 non-null  datetime64[ns]
 2   FLT_NO      14105 non-null  object        
 3   TYPE_CODE   14105 non-null  object        
 4   FROM_CITY   14105 non-null  object        
 5   TO_CITY     14105 non-null  object        
 6   CAPACITY    14105 non-null  int32         
 7   NO_PAX      14105 non-null  int32         
 8   CLASS_CODE  14105 non-null  object        
 9   BLOCK_TIME  14105 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 1.1+ MB


In [43]:
# Check the paxFrame after removing duplicates
paxFrameP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15423 entries, 0 to 163847
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CDN_NO      15423 non-null  object        
 1   DBL_DATE    15423 non-null  datetime64[ns]
 2   FLT_NO      15423 non-null  object        
 3   TYPE_CODE   15423 non-null  object        
 4   FROM_CITY   15423 non-null  object        
 5   TO_CITY     15423 non-null  object        
 6   CAPACITY    15423 non-null  int32         
 7   NO_PAX      15423 non-null  int32         
 8   CLASS_CODE  15423 non-null  object        
 9   BLOCK_TIME  15423 non-null  int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(6)
memory usage: 1.2+ MB


## Number of Passengers:

#### Total Number of Passengers:

In [44]:
paxCountP = paxFrameP['NO_PAX'].sum()
print('Total number of passenger for {} {:,}'.format(previous_month,paxCountP))

Total number of passenger for January 909,771


In [45]:
paxCount = paxFrame['NO_PAX'].sum()
print('Total number of passenger for {} {:,}'.format(current_month,paxCount))

Total number of passenger for February 699,428


#### Number of Passengers per Class for Previous Month:

In [46]:
previous_month

'January'

In [47]:
fc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'FC'].sum()
fc_weight_p = fc_count_p/paxCountP
print('First class passengers count equals {:,} PAX, and its weight is {}'.format(fc_count_p, np.round(fc_weight_p, 3)))

bc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'JC'].sum()
bc_weight_p = bc_count_p/paxCountP
print('Business class passengers count equals {:,} PAX, and its weight is {}'.format(bc_count_p, np.round(bc_weight_p, 2)))

yc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'YC'].sum()
yc_weight_p = yc_count_p/paxCountP
print('Economy class passengers count equals {:,} PAX, and its weight is {}'.format(yc_count_p, np.round(yc_weight_p, 2)))

First class passengers count equals 31 PAX, and its weight is 0.0
Business class passengers count equals 100,696 PAX, and its weight is 0.11
Economy class passengers count equals 809,044 PAX, and its weight is 0.89


##### Number of Passengers per Class for Current Month:

In [48]:
current_month

'February'

In [49]:
fc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'FC'].sum()
fc_weight = fc_count/paxCount
print('First class passengers count equals {:,} PAX, and its weight is {}'.format(fc_count, np.round(fc_weight, 3)))

bc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'JC'].sum()
bc_weight = bc_count/paxCount
print('Business class passengers count equals {:,} PAX, and its weight is {}'.format(bc_count, np.round(bc_weight, 2)))

yc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'YC'].sum()
yc_weight = yc_count/paxCount
print('Economy class passengers count equals {:,} PAX, and its weight is {}'.format(yc_count, np.round(yc_weight, 2)))

First class passengers count equals 5 PAX, and its weight is 0.0
Business class passengers count equals 73,772 PAX, and its weight is 0.11
Economy class passengers count equals 625,651 PAX, and its weight is 0.89


### Seat Capacity:

#### Total Capacity:

In [50]:
seatCapacityP = paxFrameP['CAPACITY'].sum()
print('Total capacity for domestic flights for {} equals {:,} seat.'.format(previous_month,seatCapacityP))

Total capacity for domestic flights for January equals 1,452,349 seat.


In [51]:
seatCapacity = paxFrame['CAPACITY'].sum()
print('Total capacity for domestic flights for {} equals {:,} seat.'.format(current_month,seatCapacity))

Total capacity for domestic flights for February equals 1,299,321 seat.


##### Capacity per Class Previous Month:

In [52]:
fc_capacity_p = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'FC'].sum()
print('First class seat capacity equals {:,} seat.'.format(fc_capacity_p))

bc_capacity_p  = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'JC'].sum()
print('Business class seat capacity equals {:,} seat.'.format(bc_capacity_p))

yc_capacity_p  = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'YC'].sum()
print('Economy class seat capacity equals {:,} seat.'.format(yc_capacity_p))

First class seat capacity equals 60 seat.
Business class seat capacity equals 129,782 seat.
Economy class seat capacity equals 1,322,507 seat.


##### Capacity per Class Current month:

In [53]:
fc_capacity = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'FC'].sum()
print('First class seat capacity equals {:,} seat.'.format(fc_capacity))

bc_capacity  = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'JC'].sum()
print('Business class seat capacity equals {:,} seat.'.format(bc_capacity))

yc_capacity  = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'YC'].sum()
print('Economy class seat capacity equals {:,} seat.'.format(yc_capacity))

First class seat capacity equals 24 seat.
Business class seat capacity equals 122,162 seat.
Economy class seat capacity equals 1,177,135 seat.


### Load Factor:

#### Total Load Factor:

In [54]:
loadFactor = (paxCountP/seatCapacityP)*100
print('domestic flights load factor for {} {}%'.format(previous_month, np.round(loadFactor,2)))

domestic flights load factor for January 62.64%


In [55]:
loadFactor = (paxCount/seatCapacity)*100
print('domestic flights load factor for {} {}%'.format(current_month, np.round(loadFactor,2)))

domestic flights load factor for February 53.83%


##### Load Factor per Class Previous Month:

In [56]:
fc_lf_p = (fc_count_p/fc_capacity_p)*100
print('First class load factor equals {}%'.format(np.round(fc_lf_p, 2)))

bc_lf_p = (bc_count_p/bc_capacity_p)*100
print('Business class load factor equals {}%'.format(np.round(bc_lf_p, 2)))

yc_lf_p  = (yc_count_p/yc_capacity_p)*100
print('Economy class load factor equals {}%'.format(np.round(yc_lf_p, 2)))

First class load factor equals 51.67%
Business class load factor equals 77.59%
Economy class load factor equals 61.18%


##### Load Factor per Class Current Month:

In [57]:
fc_lf = (fc_count/fc_capacity)*100
print('First class load factor equals {}%'.format(np.round(fc_lf, 2)))

bc_lf = (bc_count/bc_capacity)*100
print('Business class load factor equals {}%'.format(np.round(bc_lf, 2)))

yc_lf  = (yc_count/yc_capacity)*100
print('Economy class load factor equals {}%'.format(np.round(yc_lf, 2)))

First class load factor equals 20.83%
Business class load factor equals 60.39%
Economy class load factor equals 53.15%


# Expenses:

### Total Expenses:

In [58]:
domExpP = df_p['PRICE'][df_p['TYPE_CODE'] == 'C5'].sum()
print('The toal expenses for domestic flights for {} equals SAR {:,}'.format(previous_month, domExpP))

The toal expenses for domestic flights for January equals SAR 26,300,126


In [59]:
domExp = df['PRICE'][df['TYPE_CODE'] == 'C5'].sum()
print('The toal expenses for domestic flights for {} equals SAR {:,}'.format(current_month,domExp))

The toal expenses for domestic flights for February equals SAR 22,006,336


In [60]:
diffrence = domExp - domExpP
print('Diffrence is {:,}'.format(diffrence))

Diffrence is -4,293,790


##### Handling Charges:

In [61]:
handlingChargesP = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5')&(df_p.ITEM_CODE.isin(['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI', '#HAEE']))].sum()
handlingPerPaxP = handlingChargesP/paxCountP
print('The total handling charges for {} equals to {:,} and handling per pax equals to {}'.format(previous_month,handlingChargesP, np.round(handlingPerPaxP, 2)))

The total handling charges for January equals to 15,294,720 and handling per pax equals to 16.81


In [62]:
handlingCharges = df['PRICE'][(df['TYPE_CODE'] == 'C5')& (df.ITEM_CODE.isin(['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI', '#HAEE']))].sum()
handlingPerPax = handlingCharges/paxCount
print('The total handling charges for {} equals to {:,} and handling per pax equals to {}'.format(current_month,handlingCharges, np.round(handlingPerPax, 2)))

The total handling charges for February equals to 13,492,179 and handling per pax equals to 19.29


##### Meal Charges:

In [63]:
meal_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5')& ~(df_p.ITEM_CODE.isin(['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI', '#HAEE']))].sum()
mealPerPaxP = meal_p/paxCountP
print('The total food charges for {} equals to {:,} and food per pax equals to {}'.format(previous_month, meal_p, np.round(mealPerPaxP, 2)))

The total food charges for January equals to 11,005,406 and food per pax equals to 12.1


In [64]:
meal = df['PRICE'][(df['TYPE_CODE'] == 'C5')& ~(df.ITEM_CODE.isin(['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI', '#HAEE']))].sum()
mealPerPax = meal/paxCount
print('The total food charges for {} equals to {:,} and food per pax equals to {}'.format(current_month,  meal, np.round(mealPerPax, 2)))

The total food charges for February equals to 8,514,157 and food per pax equals to 12.17


In [65]:
diffrence_total_meal_cost = meal - meal_p
print('total diffrent in meal cost is {:,}'.format(diffrence_total_meal_cost))

total diffrent in meal cost is -2,491,249


In [66]:
diffrent_cpm = mealPerPax - mealPerPaxP
print('diffrent in CPM is {}'.format(np.round(diffrent_cpm, 2)))

diffrent in CPM is 0.08


##### Expenses per Class for Previous Month:

In [67]:
fc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'FC')].sum()
print('First class expenses equals to SAR {:,}'.format(fc_exp_p))

bc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'JC')].sum()
print('Bsiness class expenses equals to SAR {:,}'.format(bc_exp_p))

yc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'YC')].sum() - handlingChargesP
print('Economy class expenses equals to SAR {:,}'.format(yc_exp_p))

First class expenses equals to SAR 2,062
Bsiness class expenses equals to SAR 5,100,282
Economy class expenses equals to SAR 5,903,062


##### Expenses per Class for Current month:

In [68]:
fc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'FC')].sum()
print('First class expenses equals to SAR {:,}'.format(fc_exp))

bc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'JC')].sum()
print('Bsiness class expenses equals to SAR {:,}'.format(bc_exp))

yc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'YC')].sum() - handlingCharges
print('Economy class expenses equals to SAR {:,}'.format(yc_exp))

First class expenses equals to SAR 750
Bsiness class expenses equals to SAR 3,875,820
Economy class expenses equals to SAR 4,637,587


##### Cost per Passenger (CPP) for Previous Month:

In [69]:
fc_cpp_p = fc_exp_p / fc_count_p
fc_cpp_h_p = fc_cpp_p + handlingPerPaxP
print('First class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(fc_cpp_p, 2), np.round(fc_cpp_h_p, 2)))

bc_cpp_p = bc_exp_p / bc_count_p
bc_cpp_h_p = bc_cpp_p + handlingPerPaxP
print('Business class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(bc_cpp_p, 2), np.round(bc_cpp_h_p, 2)))

yc_cpp_p = yc_exp_p / yc_count_p
yc_cpp_h_p = yc_cpp_p + handlingPerPaxP
print('Economy class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(yc_cpp_p, 2), np.round(yc_cpp_h_p, 2)))

avgCppP = domExpP/paxCountP
avgCppNoHacP = avgCppP - handlingPerPaxP
print('The average CPP equals SAR {} and average meal CPP equals SAR {} .'.format(np.round(avgCppP,2), np.round(avgCppNoHacP, 2)))

First class meal cpp equals to SAR 66.52 and SAR 83.33 including handling charges.
Business class meal cpp equals to SAR 50.65 and SAR 67.46 including handling charges.
Economy class meal cpp equals to SAR 7.3 and SAR 24.11 including handling charges.
The average CPP equals SAR 28.91 and average meal CPP equals SAR 12.1 .


##### Cost per Passenger (CPP) for Current Month:

In [70]:
fc_cpp = fc_exp / fc_count
fc_cpp_h = fc_cpp + handlingPerPax
print('First class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(fc_cpp, 2), np.round(fc_cpp_h, 2)))

bc_cpp = bc_exp / bc_count
bc_cpp_h = bc_cpp + handlingPerPax
print('Business class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(bc_cpp, 2), np.round(bc_cpp_h, 2)))

yc_cpp = yc_exp / yc_count
yc_cpp_h = yc_cpp + handlingPerPax
print('Economy class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(yc_cpp, 2), np.round(yc_cpp_h, 2)))

avgCpp = domExp/paxCount
avgCppNoHac = avgCpp - handlingPerPax
print('The average CPP equals SAR {} and average meal CPP equals SAR {} .'.format(np.round(avgCpp,2), np.round(avgCppNoHac, 2)))

First class meal cpp equals to SAR 150.0 and SAR 169.29 including handling charges.
Business class meal cpp equals to SAR 52.54 and SAR 71.83 including handling charges.
Economy class meal cpp equals to SAR 7.41 and SAR 26.7 including handling charges.
The average CPP equals SAR 31.46 and average meal CPP equals SAR 12.17 .


### What are the monthly changes and their impact?

### Create Menu Frames by Service:

In [71]:
# make a copy of the invoice df and df_p
df_three = df.copy()
df_p_three = df_p.copy()

In [72]:
# Define variables for filteration  
handling_item_codes =['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI', '#HAEE']
# Flights that supposed to be beverage but follows meal policy
meal_exception = ['RUH-MED-RUH', 'RUH-YNB-RUH', 'RUH-ABT-RUH', 'RUH-BHH-RUH', 'RUH-TIF-RUH', 'MED-RUH-MED' ]
# Flights that supposed to be meal but follows beverage policy
beverage_exception = ['JED-ELQ-JED', 'JED-HAS-JED', 'JED-EAM-JED', 'JED-WAE-JED', 'JED-GIZ-JED', 'RUH-RAH-RUH' ]
# Add route column for comparsion purposes 
df_three['ROUTE'] = df_three['FROM_CITY'] + '-' + df_three['TO_CITY'] + '-' + df_three['FROM_CITY'] 
df_p_three['ROUTE'] = df_p_three['FROM_CITY'] + '-' + df_p_three['TO_CITY'] + '-' + df_p_three['FROM_CITY'] 
#unify the PO 51 and 49 to be 50
df_three.loc[((df_three.ALOTMENT =='51') | (df_three.ALOTMENT =='49')), 'ALOTMENT'] = '50'
df_p_three.loc[((df_p_three.ALOTMENT =='51') | (df_p_three.ALOTMENT =='49')), 'ALOTMENT'] = '50'
#change the PO 39 and 40
df_three.loc[(df_three.ALOTMENT =='39'), 'ALOTMENT'] = '40'
df_p_three.loc[(df_p_three.ALOTMENT =='39'), 'ALOTMENT'] = '40'
#change the PO 61 and 60
df_three.loc[(df_three.ALOTMENT =='61'), 'ALOTMENT'] = '60'
df_p_three.loc[(df_p_three.ALOTMENT =='61'), 'ALOTMENT'] = '60'

### Defining a Function that produces analysis: 

In [208]:
def cost_impact_calc(class_code, service):
    
    changes_frame = pd.DataFrame()
    
    # define query variables based on service
    if service == 'meal':
        block_time_cond_1 = (df_three.BLOCK_TIME > 89)
        block_time_cond_2 = (df_p_three.BLOCK_TIME > 89)
        not_excepted = beverage_exception
        excepted = meal_exception
    else:
        block_time_cond_1 = (df_three.BLOCK_TIME <= 89)
        block_time_cond_2 = (df_p_three.BLOCK_TIME <=  89)
        not_excepted = meal_exception
        excepted = beverage_exception
    
    if class_code  == 'YC':
        class_name = 'economy'
    else:
        class_name = 'business'
        
    # create data frames for cm and pm 
    # cm_frame data for the current month
    # pm_farme data for the previous month
    cm_frame = df_three[((block_time_cond_1) & (df_three.CLASS_CODE == class_code) &\
                              ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(not_excepted))) |\
                             ((df_three.ROUTE.isin(excepted)) &\
                              (df_three.CLASS_CODE == class_code) & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
    pm_frame = df_p_three[((block_time_cond_2) & (df_p_three.CLASS_CODE == class_code) &\
                                  ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(not_excepted)))\
                                 | ( (df_p_three.ROUTE.isin(excepted)) & (df_p_three.CLASS_CODE == class_code) &\
                                    ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]
    
    #Create another frame for the new month to calculate the impact
    cm_frame_im = df_three[((block_time_cond_1) & (df_three.CLASS_CODE == class_code) &\
                                 ~(df_three.ITEM_CODE.isin(handling_item_codes)) &\
                                 ~(df_three.ROUTE.isin(not_excepted))) | ((df_three.ROUTE.isin(excepted)) &\
                                 (df_three.CLASS_CODE == class_code) & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
    #Create another frame for the previous month to calculate the impact
    pm_frame_im = df_p_three[((block_time_cond_2) & (df_p_three.CLASS_CODE == class_code) &\
                                     ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) &\
                                     ~(df_p_three.ROUTE.isin(not_excepted))) |\
                                    ((df_p_three.ROUTE.isin(excepted)) & (df_p_three.CLASS_CODE == class_code) &\
                                     ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]
    
    cm_frame = cm_frame.drop_duplicates(subset=['DBL_DATE','SPECS_NO', 'ALOTMENT', 'UNIT_PRICE', 'ROUTE'])
    pm_frame = pm_frame.drop_duplicates(subset=['DBL_DATE','SPECS_NO', 'ALOTMENT', 'UNIT_PRICE', 'ROUTE'])

    # drop unneeded columns
    cm_frame = cm_frame.drop(columns=['CDN_NO','ETD', 'FLT_NO' ,'FROM_CITY', 'TO_CITY', 'CLASS_CODE',\
                                                'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'BLOCK_TIME'])
    pm_frame = pm_frame.drop(columns=['CDN_NO','ETD', 'FLT_NO','FROM_CITY', 'TO_CITY', 'CLASS_CODE',\
                                                    'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'BLOCK_TIME'])

    #Keep leg number 1 only 
    cm_frame = cm_frame.drop(cm_frame[cm_frame.LEGNO.isin(['2', '3', '4'])].index)
    pm_frame = pm_frame.drop(pm_frame[pm_frame.LEGNO.isin(['2', '3', '4'])].index)

    # Drop rows where unit price equals 0
    cm_frame = cm_frame.drop (cm_frame[cm_frame['UNIT_PRICE'] == 0].index)
    pm_frame = pm_frame.drop (pm_frame[pm_frame['UNIT_PRICE'] == 0].index)

    # Concat menus 
    changes_frame = pd.concat([cm_frame, pm_frame])

    # remove empty rows using the dropna() function
    changes_frame = changes_frame.dropna(subset=['SPECS_NO'])

    #drop duplicated rows
    # Add other subset 
    changes_frame = changes_frame.drop_duplicates(subset=['SPECS_NO', 'ALOTMENT', 'UNIT_PRICE', 'ROUTE'])

    
    changes_frame['UPLIFT_PERCENTAGE'] = 0
    changes_frame['CHANGE_TYPE'] = ''
    #Get the total cost of each new item
    changes_frame['TOTAL_COST'] = 0 # Add impact column to change_list
    changes_frame['CPP_IMPACT'] = 0 # Add impact column to change_list
    cost_im = 0  # place holder for total cost impact for each item
    cpp_im = 0 # place holder for CPM impact for each item
    # Loop to go through the items of the changes list and remove any item that is not a change
    # index 5 = SPEC
    # index 6 = Ratio
    # index 7 = unit price
    # index 8 = ROUTE
    for x, z in enumerate(changes_frame.values):
    
        # if the item is not available in the previous month but available in the current month "keep it" 
        if (z[5] not in pm_frame['SPECS_NO'][pm_frame.ROUTE == z[8]].values) &\
        (z[5] in cm_frame['SPECS_NO'][cm_frame.ROUTE == z[8]].values):
            
            # clasify the type of change
            changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                              (changes_frame.ALOTMENT == z[6])&\
                              (changes_frame.UNIT_PRICE == z[7])&\
                              (changes_frame.ROUTE == z[8]), 'CHANGE_TYPE'] = 'new'
            #calculate the impact of this change by removing the total cost of the change from the df, then subtracting 
            #the item cpp from the average cpp
            cost_im = cm_frame_im['PRICE'][(cm_frame_im.SPECS_NO == z[5])&\
                                           (cm_frame_im.ALOTMENT == z[6])&\
                                           (cm_frame_im.UNIT_PRICE == z[7])&\
                                           (cm_frame_im.ROUTE== z[8])].sum()
            cpp_im = cost_im/paxCount
            changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                              (changes_frame.ALOTMENT == z[6])&\
                              (changes_frame.UNIT_PRICE == z[7])&\
                              (changes_frame.ROUTE == z[8]), 'TOTAL_COST'] = cost_im
            
            changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                              (changes_frame.ALOTMENT == z[6])&\
                              (changes_frame.UNIT_PRICE == z[7])&\
                              (changes_frame.ROUTE == z[8]), 'CPP_IMPACT'] = cpp_im
            cost_im = 0
            cpp_im = 0

         # if the item is available in the previous month but not available in the current month "keep it"
        elif (z[5] not in cm_frame['SPECS_NO'][cm_frame.ROUTE == z[8]].values) &\
        (z[5] in pm_frame['SPECS_NO'][pm_frame.ROUTE == z[8]].values):
            
             # clasify the type of change
            changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                              (changes_frame.ALOTMENT == z[6])&\
                              (changes_frame.UNIT_PRICE == z[7])&\
                              (changes_frame.ROUTE == z[8]), 'CHANGE_TYPE'] = 'remove'
            
            #calculate the impact of this change by adding the total cost of the change to the df, then subtracting 
            #the average cpp from the item cpp
            cost_im_2 = -1*((pm_frame_im['PRICE'][(pm_frame_im.SPECS_NO == z[5])&\
                                                  (pm_frame_im.ALOTMENT == z[6])&\
                                                  (pm_frame_im.UNIT_PRICE == z[7])&\
                                                  (pm_frame_im.ROUTE == z[8])].sum())/paxCountP)

            cost_im = cost_im_2 * paxCount
            cpp_im =  cost_im_2

            changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                              (changes_frame.ALOTMENT == z[6])&\
                              (changes_frame.UNIT_PRICE == z[7])&\
                              (changes_frame.ROUTE == z[8]), 'TOTAL_COST'] = cost_im
            
            changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                              (changes_frame.ALOTMENT == z[6])&\
                              (changes_frame.UNIT_PRICE == z[7])&\
                              (changes_frame.ROUTE == z[8]), 'CPP_IMPACT'] = cpp_im
            cost_im = 0
            cpp_im = 0

        # delete the item if it is available in the whole previous month (100%) 
        elif (z[5] in cm_frame['SPECS_NO'][cm_frame['ROUTE'] == z[8]].values) &\
        (z[5] in pm_frame['SPECS_NO'][pm_frame['ROUTE'] == z[8]].values):

            # check if the unit price is above 0 before going through this loop
            if cm_frame['UNIT_PRICE'][(cm_frame['SPECS_NO'] == z[5])&\
                                      (cm_frame['ROUTE'] == z[8])].sum() > 0:
                
                # unique days the item has been uplifted for the current month
                alpha = cm_frame_im['DBL_DATE'][(cm_frame_im.SPECS_NO == z[5]) &\
                                                (cm_frame_im.ALOTMENT == z[6]) &\
                                                (cm_frame_im.UNIT_PRICE == z[7]) &\
                                                (cm_frame_im.ROUTE == z[8])].nunique()
                # unique days the item has been uplifted in the previous month
                bravo = pm_frame_im['DBL_DATE'][(pm_frame_im.SPECS_NO == z[5])&\
                                                (pm_frame_im.ALOTMENT == z[6])&\
                                                (pm_frame_im.UNIT_PRICE == z[7])&\
                                                (pm_frame_im.ROUTE == z[8])].nunique()
                
                # calculate the uplift percentage for the same item in both months
                uplift_percentage_a = (alpha / 30)*100
                uplift_percentage_b = (bravo / 30)*100
                

                # Setting the new value
                changes_frame.loc[(changes_frame.SPECS_NO == z[5]) &\
                                  (changes_frame.ALOTMENT == z[6])&\
                                  (changes_frame.UNIT_PRICE == z[7])&\
                                  (changes_frame.ROUTE == z[8]), 'UPLIFT_PERCENTAGE_PM']\
                = np.round(uplift_percentage_b,2)

                # keep it if it is uplifted less than 10% of the days of previuos month as new item
                # delete if it is a change in cycle
                # delete if item is uplifted in both months with less uplift in the current month
                if (uplift_percentage_b >= 10) | (uplift_percentage_a < uplift_percentage_b)| (alpha == bravo):
                    changes_frame.drop(changes_frame[(changes_frame.SPECS_NO == z[5])&\
                                                     (changes_frame.ALOTMENT == z[6])&\
                                                     (changes_frame.UNIT_PRICE == z[7])&\
                                                     (changes_frame.ROUTE == z[8])].index, inplace=True)

                # keep it if it is uplifted less than 10% of the days of previuos month as new item
                elif (uplift_percentage_b < 10) & (uplift_percentage_b > 0):
                    changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                      (changes_frame.ALOTMENT == z[6])&\
                                      (changes_frame.UNIT_PRICE == z[7])&\
                                      (changes_frame.ROUTE == z[8]), 'CHANGE_TYPE'] = 'new'
                    #calculate the impact of this change by removing the total cost of the change from the df, then subtracting 
                    #the item cpp from the average cpp
                    cost_im_cm = (cm_frame_im['PRICE'][(cm_frame_im.SPECS_NO == z[5])&\
                                                       (cm_frame_im.ALOTMENT == z[6])&\
                                                       (cm_frame_im.UNIT_PRICE == z[7])&\
                                                       (cm_frame_im.ROUTE == z[8])].sum())
                    cpp_cm = cost_im_cm / paxCount
                     
                    cost_im_pm = (pm_frame_im['PRICE'][(pm_frame_im.SPECS_NO == z[5])&\
                                                       (pm_frame_im.ALOTMENT == z[6])&\
                                                       (pm_frame_im.UNIT_PRICE == z[7])&\
                                                       (pm_frame_im.ROUTE == z[8])].sum())
                    cpp_pm = cost_im_pm / paxCountP
                    
                    cost_im = cost_im_cm
                    cpp_im = cpp_cm - cpp_pm
                    
                    changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                      (changes_frame.ALOTMENT == z[6])&\
                                      (changes_frame.UNIT_PRICE == z[7])&\
                                      (changes_frame.ROUTE == z[8]), 'TOTAL_COST'] = cost_im
                    
                    changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                      (changes_frame.ALOTMENT == z[6])&\
                                      (changes_frame.UNIT_PRICE == z[7])&\
                                      (changes_frame.ROUTE == z[8]), 'CPP_IMPACT'] = cpp_im
                    cost_im = 0
                    cpp_im = 0

                 # keep it if price has changed from previous month
                if (cm_frame['UNIT_PRICE'][(cm_frame.SPECS_NO == z[5])&\
                                           (cm_frame.ALOTMENT == z[6])&\
                                           (cm_frame.UNIT_PRICE == z[7])&\
                                           (cm_frame.ROUTE == z[8])].size>0)&\
                (pm_frame['UNIT_PRICE'][(pm_frame.SPECS_NO == z[5])&\
                                        (pm_frame.ALOTMENT == z[6])&\
                                        (pm_frame.UNIT_PRICE == z[7])&\
                                        (pm_frame.ROUTE == z[8])].size>0):
                    print(1)
                    if (cm_frame['UNIT_PRICE'][(cm_frame.SPECS_NO == z[5])&\
                                               (cm_frame.ALOTMENT == z[6])&\
                                               (cm_frame.UNIT_PRICE == z[7])&\
                                               (cm_frame.ROUTE == z[8])].unique())!=\
                    (pm_frame['UNIT_PRICE'][(pm_frame.SPECS_NO == z[5])&\
                                            (pm_frame.ALOTMENT == z[6])&\
                                            (pm_frame.UNIT_PRICE == z[7])&\
                                            (pm_frame.ROUTE == z[8])].unique()):
                        print(2)
                        changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                          (changes_frame.ALOTMENT == z[6])&\
                                          (changes_frame.UNIT_PRICE == z[7])&\
                                          (changes_frame.ROUTE == z[8]), 'CHANGE_TYPE'] = 'price'

                        cost_im =\
                        (cm_frame_im['PRICE'][(cm_frame_im['SPECS_NO'] == z[5])&\
                                              (cm_frame_im['UNIT_PRICE'] == z[7])&\
                                              (cm_frame_im['ROUTE'] == z[8])].sum())-\
                        (pm_frame_im['PRICE'][(pm_frame_im['SPECS_NO'] == z[5])&\
                                              (pm_frame_im['UNIT_PRICE']==z[7])&\
                                              (pm_frame_im['ROUTE']==z[8])].sum())
                        #calculate the impact of this change by removing the total cost of the change from the df, then subtracting 
                        #the item cpp from the average cpp
                        cpp_im = cost_im/paxCount
                        changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                                 (changes_frame.ROUTE == z[8]), 'TOTAL_COST'] = cost_im
                        changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                                 (changes_frame.ROUTE == z[8]), 'CPP_IMPACT'] = cpp_im
                        cost_im = 0
                        cpp_im = 0

             # keep it if ratio has changed from previous month
                if(cm_frame['ALOTMENT'][(cm_frame.SPECS_NO == z[5])&\
                                             (cm_frame.ALOTMENT == z[6])&\
                                             (cm_frame.ROUTE == z[8])].size>0)&\
                (pm_frame['ALOTMENT'][(pm_frame.SPECS_NO == z[5])&\
                                             (pm_frame.ALOTMENT == z[6])&\
                                             (pm_frame.ROUTE == z[8])].size>0):

                    if(cm_frame['ALOTMENT'][(cm_frame['SPECS_NO'] == z[5])&\
                                                 (cm_frame['ALOTMENT'] == z[6])&\
                                                 (cm_frame['ROUTE'] == z[8])].unique())!=\
                    (pm_frame['ALOTMENT'][(pm_frame['SPECS_NO'] == z[5])&\
                                                 (pm_frame['ALOTMENT'] == z[6])&\
                                                 (pm_frame['ROUTE'] == z[8])].unique()):

                        changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                                 (changes_frame.SPECS_NO == z[8]), 'CHANGE_TYPE'] = 'ratio'

                        cost_im = (cm_frame_im['PRICE'][(cm_frame_im['SPECS_NO'] == z[5])&\
                                                                  (cm_frame_im['ALOTMENT'] == z[6])&\
                                                                  (cm_frame_im['ROUTE'] == z[8])].sum())-\
                        (pm_frame_im['PRICE'][(pm_frame_im['SPECS_NO'] == z[5])&\
                                                     (pm_frame_im['ALOTMENT']==z[6])&\
                                                     (pm_frame_im['ROUTE']==z[8])].sum())
                        #calculate the impact of this change by removing the total cost of the change from the df, then subtracting 
                        #the item cpp from the average cpp
                        cpp_im = cost_im/paxCount
                        changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                                 (changes_frame.ROUTE == z[8]), 'TOTAL_COST'] = cost_im
                        changes_frame.loc[(changes_frame.SPECS_NO == z[5])&\
                                                 (changes_frame.ROUTE == z[8]), 'CPP_IMPACT'] = cpp_im
                        cost_im = 0
                        cpp_im = 0

    #round up the cpp_impact
    changes_frame['CPP_IMPACT'] = np.round(changes_frame['CPP_IMPACT'], 2)
    
    #round up the total cost
    changes_frame['TOTAL_COST'] = np.round(changes_frame['TOTAL_COST'])

    # Remove duplicates rows
    cm_frame.drop_duplicates(subset=['SPECS_NO'], inplace= True)
    pm_frame.drop_duplicates(subset=['SPECS_NO'], inplace= True)

    #Reset index values 
    cm_frame.reset_index(drop= True, inplace = True)
    pm_frame.reset_index(drop= True, inplace = True)
    changes_frame.reset_index(drop= True, inplace = True)

    #drop uneeded coloumns
    changes_frame.drop(columns=['LEGNO', 'ITEM_CODE'], inplace=True)
    sleep(1) # Time in seconds

    if changes_frame['ITEM_DESCRIPTION'].count() > 0:
        # print previous month dataframe
        print('{} menu:'.format(previous_month))
        display(pm_frame)
        # print current month dataframe
        print('{} menu:'.format(current_month))
        display(cm_frame)
        print('The following is the changes and impact breakdown:')
        display(changes_frame)
        print('Total cost impact on meal flights {} equals SAR {:,} and its impact on CPP equals SAR {:,}'\
              .format(class_name, np.round(changes_frame['TOTAL_COST'].sum(), 2), np.round(changes_frame['CPP_IMPACT'].sum(), 2)))

    else:
        # print previous month dataframe
        print('Previous month menu:')
        display(pm_frame)
        # print current month dataframe
        print('Current month menu:')
        display(cm_frame)
        print('No changes observed')
        
    return changes_frame, pm_frame_im, cm_frame_im

### Beverage Flights:

<a id='yc_bev'></a>
#### YC beverage flight impact:

In [209]:
yc_bev_changes, yc_bev_pm, yc_bev_cm  = cost_impact_calc('YC', 'bev')

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
January menu:


,DBL_DATE,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE
0,2021-01-01,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41,DMM-RUH-DMM
1,2021-01-01,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38,DMM-RUH-DMM
2,2021-01-23,1,SD1802,WATER MINERAL HAYAT (1.5LTR),BTL,41288,NaN,1.13,DMM-RUH-DMM
3,2021-01-11,1,#SDICE1,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52,JED-AHB-JED
4,2021-01-26,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,NaN,1.38,RUH-DMM-RUH
5,2021-01-30,1,DP06A,DP06A-ICE CUBES 5KG/BAG,BAG,36622,1-60/1,10.35,RUH-DMM-RUH
6,2021-01-30,1,SD02,SD02-PEPSI COLA,CAN,35277,1-60/2,3.17,RUH-DMM-RUH
7,2021-01-30,1,SD03,SD03-7UP (355ML),CAN,35279,1-60/2,3.18,RUH-DMM-RUH
8,2021-01-30,1,SD11,SD11-DIET PEPSI,CAN,35278,1-60/1,3.18,RUH-DMM-RUH
9,2021-01-30,1,SD12,SD12-DIET 7UP,CAN,35280,1-60/1,3.18,RUH-DMM-RUH


February menu:


,DBL_DATE,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE
0,2021-02-01,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41,DMM-RUH-DMM
1,2021-02-01,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38,DMM-RUH-DMM
2,2021-02-27,1,#SDICE1,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52,JED-GIZ-JED
3,2021-02-28,1,SD1801,"TSC, SD1801-WATER CUP W/O LOGO HAYAT (125ML)",CUP,41289,100,0.41,MED-JED-MED
4,2021-02-28,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.38,MED-JED-MED
5,2021-02-28,1,SNYIC2041,S/W RECT TURKEY SMOKED-PEPPERS/THREE CHEESE-,PO,42855,100,10.97,MED-JED-MED


The following is the changes and impact breakdown:


,DBL_DATE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE,UPLIFT_PERCENTAGE,CHANGE_TYPE,TOTAL_COST,CPP_IMPACT,UPLIFT_PERCENTAGE_PM
0,2021-02-27,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52,JED-GIZ-JED,0,,0.0,0.0,NaN
1,2021-02-07,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52,JED-AHB-JED,0,,0.0,0.0,NaN
2,2021-02-10,WATER MINERAL HAYAT (200ML),BTL,32671,NaN,0.41,JED-AHB-JED,0,,0.0,0.0,NaN
3,2021-02-10,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,NaN,1.38,JED-AHB-JED,0,,0.0,0.0,NaN
4,2021-02-26,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52,JED-EAM-JED,0,,0.0,0.0,NaN
5,2021-02-28,"TSC, SD1801-WATER CUP W/O LOGO HAYAT (125ML)",CUP,41289,100,0.41,MED-JED-MED,0,new,36.0,0.0,NaN
6,2021-02-28,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.38,MED-JED-MED,0,new,124.0,0.0,NaN
7,2021-02-28,S/W RECT TURKEY SMOKED-PEPPERS/THREE CHEESE-,PO,42855,100,10.97,MED-JED-MED,0,new,987.0,0.0,NaN
8,2021-02-06,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52,RUH-AQI-RUH,0,,0.0,0.0,NaN
9,2021-02-11,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41,AHB-JED-AHB,0,new,43.0,0.0,NaN


Total cost impact on meal flights economy equals SAR 1,168.0 and its impact on CPP equals SAR 0.0


<a id='bc_bev'></a>
##### BC beverage flight impact:

In [206]:
bc_bev_changes, bc_bev_pm, bc_bev_cm =cost_impact_calc('JC', 'bev')

January menu:


,DBL_DATE,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE
0,2021-01-01,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41,DMM-RUH-DMM
1,2021-01-01,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38,DMM-RUH-DMM
2,2021-01-01,1,SD02,SD02-PEPSI COLA,CAN,35277,1-10/5,3.17,DMM-RUH-DMM
3,2021-01-01,1,SD03,SD03-7UP (355ML),CAN,35279,1-10/5,3.18,DMM-RUH-DMM
4,2021-01-01,1,SD11,SD11-DIET PEPSI,CAN,35278,1-10/3,3.18,DMM-RUH-DMM
5,2021-01-01,1,SD12,SD12-DIET 7UP,CAN,35280,1-10/3,3.18,DMM-RUH-DMM


February menu:


,DBL_DATE,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE
0,2021-02-01,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.410000,DMM-RUH-DMM
1,2021-02-01,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.380000,DMM-RUH-DMM
2,2021-02-01,1,SD02,SD02-PEPSI COLA,CN,35277,1-10/5,3.170000,DMM-RUH-DMM
3,2021-02-01,1,SD03,SD03-7UP (355ML),CN,35279,1-10/5,3.180000,DMM-RUH-DMM
4,2021-02-01,1,SD11,SD11-DIET PEPSI,CN,35278,1-10/3,3.180000,DMM-RUH-DMM
5,2021-02-01,1,SD12,SD12-DIET 7UP,CN,35280,1-10/3,3.180000,DMM-RUH-DMM
6,2021-02-28,1,DSFFI1409,DSFFI1409-CAKE CHEESE-RASPBERRY/FRUITS,PO,33431,100,9.290000,MED-JED-MED
7,2021-02-28,1,ENFIM2030,"ENT, ENFIM2030-LAMB CHOP ROASTED PEPPERCRUST W/ ROASTED TOMATO SAUCE",PO,42974,40,76.019997,MED-JED-MED
8,2021-02-28,1,ENFIP2022,"ENT, ENFIP2022-CHICKEN ZORBIAN (RECT PLATE)",PO,42939,60,13.900000,MED-JED-MED
9,2021-02-28,1,HDVIA1811,ARABIC MEZZEH (HUMMUS/MOUTABAL/LABNEH ),PO,42078,100,6.250000,MED-JED-MED


The following is the changes and impact breakdown:


,DBL_DATE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE,UPLIFT_PERCENTAGE,CHANGE_TYPE,TOTAL_COST,CPP_IMPACT,UPLIFT_PERCENTAGE_PM
0,2021-02-28,DSFFI1409-CAKE CHEESE-RASPBERRY/FRUITS,PO,33431,100,9.290000,MED-JED-MED,0,new,92,0.0,NaN
1,2021-02-28,"ENT, ENFIM2030-LAMB CHOP ROASTED PEPPERCRUST W/ ROASTED TOMATO SAUCE",PO,42974,40,76.019997,MED-JED-MED,0,new,304,0.0,NaN
2,2021-02-28,"ENT, ENFIP2022-CHICKEN ZORBIAN (RECT PLATE)",PO,42939,60,13.900000,MED-JED-MED,0,new,83,0.0,NaN
3,2021-02-28,ARABIC MEZZEH (HUMMUS/MOUTABAL/LABNEH ),PO,42078,100,6.250000,MED-JED-MED,0,new,62,0.0,NaN
4,2021-02-28,WATER MINERAL HAYAT (330ML),BTL,36798,100,0.540000,MED-JED-MED,0,new,5,0.0,NaN
5,2021-02-28,ARABIC BREAD BROWN MINI (3PCS/PKT),PKT,41329,100,0.880000,MED-JED-MED,0,new,8,0.0,NaN
6,2021-02-28,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.380000,MED-JED-MED,0,new,13,0.0,NaN
7,2021-02-28,OIL OLIVE ORGANIC EX VIRGIN AL JOUF (12ML),BTL,43001,100,1.930000,MED-JED-MED,0,new,19,0.0,NaN
8,2021-02-28,CHOCOLATE VALRHONA CARRES ASSORTED (5GR),PC,43003,100,1.930000,MED-JED-MED,0,new,19,0.0,NaN
9,2021-02-28,DP01B-COFFEE NESCAFE 2GRM,SCHT,35257,1-12/4,0.690000,MED-JED-MED,0,new,2,0.0,NaN


Total cost impact on meal flights business equals SAR 753 and its impact on CPP equals SAR 0.0


### Meal Flights:

<a id='yc_meal'></a>
#### YC meal flights impact:

In [210]:
yc_meal_changes, yc_meal_pm, yc_meal_cm  = cost_impact_calc('YC', 'meal')

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
January menu:


,DBL_DATE,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE
0,2021-01-01,1,SD1801,"TSC, SD1801-WATER CUP W/O LOGO HAYAT (125ML)",CUP,41289,100,0.41,DMM-JED-DMM
1,2021-01-01,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.38,DMM-JED-DMM
2,2021-01-01,1,SNYIC2040,S/W RECT CHICKEN SIXTY FIVE/THREE CHEESE-MAYONNAISE,PO,42854,100,8.16,DMM-JED-DMM
3,2021-01-01,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38,DMM-JED-DMM
4,2021-01-01,1,TL1609,MUFFIN DATES 90GR WRAPPED,PC,40620,100,3.31,DMM-JED-DMM
5,2021-01-01,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41,DMM-JED-DMM
6,2021-01-02,1,SNYIC2041,S/W RECT TURKEY SMOKED-PEPPERS/THREE CHEESE-,PO,42855,100,10.97,DMM-JED-DMM
7,2021-01-02,1,SNYIC2038,S/W CROISSANT THREE CHEESE-TOMATO SUNDRIED SALAD,PO,42856,100,9.21,DMM-JED-DMM
8,2021-01-09,1,SNYIC2021,S/W RECT CHICKEN SHAWARMA/THREE CHEESE-MAYONNAISE WRAPPED,PO,42580,100,8.40,DMM-JED-DMM
9,2021-01-09,1,TL1611,MUFFIN VANILLA 90GR WRAPPED,PC,40844,100,3.06,DMM-JED-DMM


February menu:


,DBL_DATE,LEGNO,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE
0,2021-02-01,1,SD1801,"TSC, SD1801-WATER CUP W/O LOGO HAYAT (125ML)",CUP,41289,100,0.41,DMM-JED-DMM
1,2021-02-01,1,SD2033,JUICE APPLE AL SAFI (180ML),BTL,42829,100,1.38,DMM-JED-DMM
2,2021-02-01,1,SNYIC2041,S/W RECT TURKEY SMOKED-PEPPERS/THREE CHEESE-,PO,42855,100,10.97,DMM-JED-DMM
3,2021-02-01,1,SD2034,"BEV, SD2034-JUICE ORANGE AL SAFI (180ML)",BTL,42828,100,1.38,DMM-JED-DMM
4,2021-02-01,1,TL1608,MUFFIN WHOLEMEAL-HONEY 90GR WRAPPED,PC,40619,100,3.31,DMM-JED-DMM
5,2021-02-01,1,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,100,0.41,DMM-JED-DMM
6,2021-02-06,1,SNYIC2021,S/W RECT CHICKEN SHAWARMA/THREE CHEESE-MAYONNAISE WRAPPED,PO,42580,100,8.40,DMM-JED-DMM
7,2021-02-06,1,TL1611,MUFFIN VANILLA 90GR WRAPPED,PC,40844,100,3.06,DMM-JED-DMM
8,2021-02-06,1,SD1803,WATER MINERAL HAYAT (330ML),BTL,36798,NaN,0.54,DMM-JED-DMM
9,2021-02-13,1,SNYIC2023,S/W RECT TUNA-MAYONNAISE-PEPPERS/LABNEH-WALNUT-ZATAR WRAPPED,PO,42581,100,8.71,DMM-JED-DMM


The following is the changes and impact breakdown:


,DBL_DATE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE,UPLIFT_PERCENTAGE,CHANGE_TYPE,TOTAL_COST,CPP_IMPACT,UPLIFT_PERCENTAGE_PM
0,2021-02-01,MUFFIN WHOLEMEAL-HONEY 90GR WRAPPED,PC,40619,100,3.31,DMM-JED-DMM,0,new,2805.0,0.00,6.67
1,2021-02-06,JUICE APPLE AL SAFI (180ML),BTL,42829,NaN,1.38,DMM-JED-DMM,0,,0.0,0.00,NaN
2,2021-02-06,WATER MINERAL HAYAT (330ML),BTL,36798,NaN,0.54,DMM-JED-DMM,0,,0.0,0.00,NaN
3,2021-02-26,#SDICE1-DRY ICE PACKET,PC,37322,NaN,2.52,DMM-JED-DMM,0,,0.0,0.00,NaN
4,2021-02-01,MUFFIN WHOLEMEAL-HONEY 90GR WRAPPED,PC,40619,100,3.31,DMM-AHB-DMM,0,new,3711.0,0.00,6.67
5,2021-02-03,MUFFIN WHOLEMEAL-HONEY 90GR WRAPPED,PC,40619,100,3.31,DMM-EAM-DMM,0,new,769.0,0.00,3.33
6,2021-02-06,S/W RECT CHICKEN SHAWARMA/THREE CHEESE-MAYONNAISE WRAPPED,PO,42580,100,8.40,DMM-TUU-DMM,0,new,1554.0,0.00,NaN
7,2021-02-01,DP07B-COFFEE DECAFEINATED 2GRM,SCHT,35245,1-90/5,0.78,JED-RUH-JED,0,new,1634.0,0.00,3.33
8,2021-02-01,DP1412-SUGAR SWEETENER (SWEEVA),SCHT,38019,1-90/10,0.14,JED-RUH-JED,0,new,478.0,0.00,3.33
9,2021-02-01,DP20-TEA BAG DILMAH (25PCS/PKT),PKT,36340,1-90/2,5.60,JED-RUH-JED,0,new,5236.0,0.01,3.33


Total cost impact on meal flights economy equals SAR 34,812.0 and its impact on CPP equals SAR -0.01


In [196]:
yc_meal_cm[(yc_meal_cm.SPECS_NO == '32671') & (yc_meal_cm.ROUTE == 'RUH-GIZ-RUH')]

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE,BLOCK_TIME,ROUTE
111228,2110920010032,2021-02-21,1779,05:30,RUH,GIZ,1,YC,132,71,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,NaN,0.41,31,C5,110,RUH-GIZ-RUH


In [197]:
yc_meal_pm[(yc_meal_pm.SPECS_NO == '32671') & (yc_meal_pm.ROUTE == 'RUH-GIZ-RUH')]

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE,BLOCK_TIME,ROUTE
134934,2110920005603,2021-01-30,1803,23:55,RUH,GIZ,1,YC,132,87,SD1825,WATER MINERAL HAYAT (200ML),BTL,32671,NaN,0.41,41,C5,110,RUH-GIZ-RUH


<a id='bc_meal'></a>
#### BC meal flights impact:

In [ ]:
bc_meal_changes, bc_meal_pm, bc_meal_cm = cost_impact_calc('JC', 'meal')

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [199]:
bc_meal_changes[(bc_meal_changes.SPECS_NO == '42592') & (bc_meal_changes.ROUTE == 'RUH-MED-RUH')]

,DBL_DATE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,ROUTE,UPLIFT_PERCENTAGE,CHANGE_TYPE,TOTAL_COST,CPP_IMPACT
70,2021-02-15,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,27.84,RUH-MED-RUH,0.0,,0.0,0.0


In [200]:
bc_meal_cm[(bc_meal_cm.SPECS_NO == '42592') & (bc_meal_cm.ROUTE == 'RUH-MED-RUH')]

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE,BLOCK_TIME,ROUTE
74183,2110920008473,2021-02-13,1467,00:30,RUH,MED,1,JC,12,8,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,88,C5,95,RUH-MED-RUH
74209,2110920008474,2021-02-13,1466,00:30,RUH,MED,1,JC,12,9,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,118,C5,95,RUH-MED-RUH
74235,2110920008478,2021-02-13,1469,02:55,RUH,MED,1,JC,12,1,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,29,C5,95,RUH-MED-RUH
74347,2110920008557,2021-02-13,1465,16:50,RUH,MED,1,JC,12,12,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,148,C5,95,RUH-MED-RUH
74373,2110920008644,2021-02-14,1467,00:30,RUH,MED,1,JC,12,5,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,59,C5,95,RUH-MED-RUH
74399,2110920008645,2021-02-14,1466,00:30,RUH,MED,1,JC,12,12,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,148,C5,95,RUH-MED-RUH
74565,2110920008761,2021-02-14,1465,16:50,RUH,MED,1,JC,12,7,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,88,C5,95,RUH-MED-RUH
74591,2110920008762,2021-02-14,1464,16:50,RUH,MED,1,JC,12,12,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,148,C5,95,RUH-MED-RUH
74617,2110920008842,2021-02-15,1469,02:55,RUH,MED,1,JC,12,2,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,29,C5,95,RUH-MED-RUH
74675,2110920008906,2021-02-15,1465,16:50,RUH,MED,1,JC,20,11,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,118,C5,95,RUH-MED-RUH


In [201]:
bc_meal_pm[(bc_meal_pm.SPECS_NO == '42592') & (bc_meal_pm.ROUTE == 'RUH-MED-RUH')]

,CDN_NO,DBL_DATE,FLT_NO,ETD,FROM_CITY,TO_CITY,LEGNO,CLASS_CODE,CAPACITY,NO_PAX,ITEM_CODE,ITEM_DESCRIPTION,UOM,SPECS_NO,ALOTMENT,UNIT_PRICE,PRICE,TYPE_CODE,BLOCK_TIME,ROUTE
85907,2110920002562,2021-01-16,1467,00:30,RUH,MED,1,JC,20,8,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,88,C5,95,RUH-MED-RUH
85933,2110920002567,2021-01-16,1466,00:30,RUH,MED,1,JC,20,20,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,236,C5,95,RUH-MED-RUH
85959,2110920002574,2021-01-16,1469,02:55,RUH,MED,1,JC,12,7,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,88,C5,95,RUH-MED-RUH
86093,2110920002696,2021-01-16,1465,16:50,RUH,MED,1,JC,20,20,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,236,C5,95,RUH-MED-RUH
86119,2110920002697,2021-01-16,1464,16:50,RUH,MED,1,JC,20,20,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,236,C5,95,RUH-MED-RUH
86145,2110920002773,2021-01-17,1467,00:30,RUH,MED,1,JC,12,12,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,148,C5,95,RUH-MED-RUH
86171,2110920002774,2021-01-17,1466,00:30,RUH,MED,1,JC,12,12,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,148,C5,95,RUH-MED-RUH
86197,2110920002778,2021-01-17,1469,02:55,RUH,MED,1,JC,20,5,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,59,C5,95,RUH-MED-RUH
86358,2110920002841,2021-01-17,1465,16:50,RUH,MED,1,JC,20,14,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,148,C5,95,RUH-MED-RUH
86384,2110920002842,2021-01-17,1464,16:50,RUH,MED,1,JC,20,20,ENFIM2021,LAMB SHANK TAGINE MOROCCAN,PO,42592,40,29.620001,236,C5,95,RUH-MED-RUH


## Summary

### List of Changes:

In [202]:
total_cpm_impact = yc_bev_changes['CPP_IMPACT'].sum() + bc_bev_changes['CPP_IMPACT'].sum()\
+ yc_meal_changes['CPP_IMPACT'].sum() + bc_meal_changes['CPP_IMPACT'].sum()
print(total_cpm_impact)

0.2900000000000005


In [203]:
actual_cpm_impact = avgCppNoHac - avgCppNoHacP
actual_cpm_impact

0.07613162046548894

In [160]:
# create excel writer object
writer = pd.ExcelWriter('changes_lists.xlsx')
# write dataframe to excel
yc_bev_changes.to_excel(writer, 'yc_bev_changes')
bc_bev_changes.to_excel(writer, 'bc_bev_changes')
yc_meal_changes.to_excel(writer,'yc_meal_changes')
bc_meal_changes.to_excel(writer, 'bc_meal_changes')
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
